# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos:   Íñigo Ballester Gúrpide<br>
url: https://drive.google.com/file/d/1VtT618nBKbn9HzpBHFjJW0OnnJuxQGIF/view?usp=sharing
<br>
Colab: https://colab.research.google.com/drive/1VtT618nBKbn9HzpBHFjJW0OnnJuxQGIF
<br>
Github: https://github.com/ibagur/03MIAR-algoritmos-de-optimizacion-2021/tree/main/SEMINARIO<br>
## Problema:
# 1. Sesiones de doblaje <br>

## Descripción del problema:(copiar enunciado)



(*) La respuesta es obligatoria





                                        

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que se gasto por los servicios de los actores de doblaje sea el menor posible.

## (*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>



## ¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.




Respuesta

No se nos dan detalles con respecto a la duración temporal de las tomas, por lo que si eliminamos la restricción del máximo 6 tomas por día podemos entoncer asumir que el total de las 30 tomas podría hacerse en un solo día. En tal caso, y como en principio el orden de las tomas no nos importa, el número total de posibilidades (que de ahora en adelante llamaré 'plan de trabajo') será **1**. Básicamente ese día asisten todos los actores al estudio de grabación y ruedan las 30 tomas. El orden de grabación de las tomas es indiferente con tal de hacer las 30 tomas, ya que el gasto es el mismo (el sueldo de los 10 actores en ese día).

Si tenemos en cuenta la restricción de máximo 6 tomas diarias, la cosa ya se nos complica un poco...En principio voy a asumir que cada día se realizaría ese máximo número de tomas (6), por lo que necesitaríamos 5 días de trabajo para completar las 30 tomas. El problema se transforma entonces en lo siguiente: **¿De cuántas maneras es posible agrupar 30 tomas diferentes en 5 sesiones de 6 tomas de tal manera que cubra las 30 diferentes tomas en los 5 días?** O lo que es lo mismo, **¿cuántos planes de trabajo diferentes puedo tener, donde un plan de trabajo es 5 días a 6 tomas cada día, obviamente sin repetir tomas?**

En un cálculo combinatorio inicial tendríamos lo siguiente:

$$ {30\choose 6}{24\choose 6}{18\choose 6}{12\choose 6}{6\choose 6} = \frac{30!}{(6!)^{5}} $$

Pero hay que tener en cuenta que el orden de las tomas dentro de un *plan de trabajo* (conjunto de tomas a lo largo de los 5 días) nos es indiferente, por lo que para eliminar ese *sobre-conteo*, divido el resultado anterior por el número de ordenamientos en que podríamos ordenar los sets de 6 tomas a lo largo de 5 días, o sea , 5!:

$$ N_{planes} = \frac{30!}{(6!)^{5}{5!}} = 1.142395e+16 $$



## Modelo para el espacio de planes<br>
## (*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


Para modelizar los datos del problema usaré una serie de estructuras de array:

* Array **tomas**: contendrá los 30 tipos de *tomas* diferentes y en cada toma los detalles de los actores que participan en ellas:
    * El índice *tomas[:,0]* del array numpy me servirá para identificar las tomás específicas.
    * La presencia o no de un actor determinado se determina tal como viene definida en el problema (valor 1 ó 0)

````
tomas = array([[ 1,  1,  1,  1,  1,  1,  0,  0,  0,  0,  0],
       [ 2,  0,  0,  1,  1,  1,  0,  0,  0,  0,  0],
       [ 3,  0,  1,  0,  0,  1,  0,  1,  0,  0,  0],
       [ 4,  1,  1,  0,  0,  0,  0,  1,  1,  0,  0],
       [ 5,  0,  1,  0,  1,  0,  0,  0,  1,  0,  0],
       ...
````

* Array **sesion**: Se tratará en realidad de un array dinámico obtenido al ir explorando las diferentes combinaciones de *sesiones de 6 tomas* (sin repetición y sin importar el orden) a partir del total de *tomas* disponibles cada día (30 tomas disponibles el 1er día, 24 el 2º, etc.). Las diferentes combinaciones de *6 tomas* se irán obteniendo de manera recursiva usando una estructura de árbol de nodos no binario y para cada combinación se evaluará su coste y se almacenará la que en ese momento tenga el menor coste. Consiste en un array 1D, con una fila (correspondiente a  un día de rodaje) y en cada fila los índices que indentifican las 6 tomas que se rodarán ese día. Por ejemplo, la siguiente sesión contiene las tomas 1, 2, 3, 4, 5 y 6:

````
sesion = array([ 1  2  3  4  5  6])

````
* Array **plan**: Almacenará los *planes de rodaje* viables, que consistirán en 5 sesiones consecutivas (una por día), cada una de ellas con 6 tomas diferentes. Consiste en un array 2D con 5 filas (una por sesión):

````
plan = array ([[ 1  2  3  4  5  6]
 [ 7  8  9 10 11 12]
 [13 14 17 18 20 27]
 [15 16 22 25 28 30]
 [19 21 23 24 26 29]])
````

Para la búsqueda de las combinaciones de sesiones, la función usará una búsqueda en árbol no binario. Dicha técnica puede usarse también en el algoritmo de fuerza bruta que propondré, aunque por simplicidad en dicho caso usaré una serie de bucles anidados.

## Según el modelo para el espacio de planes<br>
## (*)¿Cual es la función objetivo?

## (*)¿Es un problema de maximización o minimización?

**La función objectivo** es una función lineal con el coste combinado del número de actores con los que contaremos cada día para el doblaje de 6 tomas a lo largo de los 5 días (a fin de completar el trabajo)

$$ Total\ Cost = \min \left\{ \sum_{i=1}^{5} kn_{i}\right\}$$

donde $i$ es el número de días, $k$ es el coste unitario por actor y día (idéntico en nuestro caso para todos los actores y días), y $n_{i}$ el número de actores necesarios para completar una sesión diaria (de *6 tomas*)

**Se trata de un problema de minimización** de esa función de coste. Nos interesa cubrir las 6 tomas de cada día con el número mínimo de actores con el objetivo de que el gasto total incurrido sea el menor posible.

## Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta

El algoritmo por fuerza bruta intentará replicar el proceso de obtención de *planes de trabajo* descrito anteriormente, en el que se van realizando sucesivas combinaciones de las sesiones de tomas y evaluando el coste de los planes de trabajo conformados por los diferentes conjuntos de sesiones.

Básicamente el algorimo lo que hará será primeramente generar todas las combinaciones de 6 tomas de las 30 existentes, es decir, las posibles *sesiones* de rodaje de *6 tomas* que podemos conformar el 1er día. En un bucle, para cada una de esas *sesiones* se generarán las combinaciones de 6 tomas entre las tomas restantes. Es decir, para el 2º día serían 24 tomas disponibles ya que no contaríamos con la sesión de 6 tomas que hemos escogido. Y así sucesivamente hasta completar los 5 días (y agotar el grupo de 30 tomas inicial). El algoritmo puede describirse mediante una serie de 5 bucles anidados o mediante una función recursiva que se detenga al llegar al 5º día, de tal forma que se vayan conformando los diferentes planes de rodaje de acuerdo al producto cartesiano de todas las combinaciones sucesivas:

$$ {30\choose 6}{24\choose 6}{18\choose 6}{12\choose 6}{6\choose 6} $$

El número total de combinaciones posibles se va reduciendo a medida que avanzan los días y tenemos menos tomas disponibles (ya que hemos ido usando tomas y no es posible repetirlas), por lo que a priori no sería muy costoso computacionalmente el calcularlas por separado. El problema viene al tener que usar bucles anidados para ir conformando los diferentes *planes de trabajo* viables, ya que en ese caso el número total viene de la multiplicación de dichas cantidades tal y como he detallado al comienzo de la actividad:

$$ 593775 * 134596 * 18564 * 924 * 1 = 1.142395e+16$$


Procederé a implementar el algoritmo de fuerza bruta usando bucles anidados para la función principal que construye los planes. Dicha función podría implementarse también de manera recursiva, aunque por simplicidad conceptual usaré los bucles anidados (aunque la función que genera las combinaciones de cada día si hace uso de la recursividad).

Lo primero que haré sera cargar el set de datos de las escenas y actores en un array numpy desde un archivo CSV que he exportado desde la fuente original y crearé el *Array 1: tomas*:

In [ ]:
# Importo las librerías que me harán falta
import numpy as np
import random  
import time
from itertools import combinations
import math

In [ ]:
# Leo el archivo con los datos de doblaje y las tomas posibles
tomas = np.genfromtxt('datos_doblaje.csv', delimiter=',', skip_header = 1, dtype='uint8')

Crearé ahora una serie de funciones auxiliares. La primera es *get_indices_sesion*, que me proporciona los índices identificativos de las tomas en una sesión determinada y que utilizaré para retirar las tomas de esa de sesión del grupo de tomas disponibles para el siguiente día:

In [ ]:
# Funcion para obtener los indices ID de las tomas en una sesión
def get_indices_sesion(sesion, tomas):
    indices = tomas[sesion,0]
    return indices

Las otra función auxiliar que usaré es *sesion_cost* que me calcula el coste de una sesión de 6 tomas. Para ello realiza un OR entre todas las tomas que conformar dicha sesión, a fin de obtener la lista de actores que participarán en la sesión de ese día (en una u otra toma), y a partir de ahí realiza la suma (asumiendo coste unitario del salario diario de un actor). Como complemento, defino también la función *plan_cost* que, haciendo uso de *sesion_cost*, calcula el coste total de un plan de rodaje de 5 sesiones de 6 tomas cada una:

In [ ]:
# Función que me calcula el coste total de una sesión diaria de 6 tomas
# sesion --> lista con los indices de las 6 tomas seleccionadas en esa sesión
# tomas -> array que contiene todas las tomas restantes (originalmente las 30)
def sesion_cost(sesion, tomas):
	# transformo la lista de ID de tomas en un array numérico
	sesion_arr = np.array(sesion)
	# sustraigo 1 para obtener el indice sobre el array numpy haciendo un broadcasting
	sesion_arr = sesion_arr - 1
	# selecciona del array tomas las tomas de la sesión dada, y aplica un OR a todas las columnas a fin de obtener una lista con todos los actores de ese día, y simplemente hace la suma (asumimos el coste unitario por actor)
	cost = np.sum(tomas[sesion_arr,1:].any(axis=0))
	return cost

# Función que calcula el coste total salarial de un plan de 5 sesiones de 6 tomas
def plan_cost(plan, tomas):
	coste_total = 0
	for sesion in plan:
		coste = sesion_cost(sesion, tomas)
		coste_total += coste
	return coste_total

Teniendo ya creadas las diversas funciones auxiliares y basándome en lo descrito anteriormente, desarrollo bajo python un posible algoritmo de fuerza bruta que iría creando cada uno de los posibles planes de rodaje (planes de 5 sesiones de 6 tomas distintas) y evaluando el coste total de cada uno de ellos. Hago uso de la función *combinations* del paquete *itertools* para ir generando las diferentes combinaciones de tomas que iré necesitando en cada día:

In [ ]:
def Mejor_Plan_rodaje(tomas):
    start_time = time.time()
    # contador combinaciones
    i = 0
    # array que contendrá todas las tomas y el status de cada actor
    grupo_tomas = tomas  
    # Por defecto tendra el mayor coste posible (salarios de todos los actores todos los dias)
    mejor_coste_plan = 50
    # número de tomas en una sesión diaria
    n = 6
    # genero todas las sesiones de 6 tomas posibles para el 1er día
    sesiones = list(combinations(list(range(len(grupo_tomas))),n))
    for sesion in sesiones:
        # almaceno los ID de las tomas de esa sesion para no perderlos
        sesion_indices = get_indices_sesion(sesion, grupo_tomas)
        # calculo y almaceno el coste de la sesión
        coste_sesion = sesion_cost(sesion_indices, tomas)   
        # sustraigo las tomas de la sesion del conjunto de tomas existente
        grupo_tomas_2 = np.delete(grupo_tomas, sesion, axis=0)
        # calculo las combinaciones de tomas disponibles para el siguiente día
        sesiones_2 = list(combinations(list(range(len(grupo_tomas_2))),n))
        # ídem para el 2º día
        for sesion_2 in sesiones_2:
            sesion_indices_2 = get_indices_sesion(sesion_2, grupo_tomas_2)
            coste_sesion_2 = sesion_cost(sesion_indices_2, tomas)    
            grupo_tomas_3 = np.delete(grupo_tomas_2, sesion_2, axis=0)
            sesiones_3 = list(combinations(list(range(len(grupo_tomas_3))),n))
            # ídem para el 3º día
            for sesion_3 in sesiones_3:
                sesion_indices_3 = get_indices_sesion(sesion_3, grupo_tomas_3)
                coste_sesion_3 = sesion_cost(sesion_indices_3, tomas)   
                grupo_tomas_4 = np.delete(grupo_tomas_3, sesion_3, axis=0)              
                sesiones_4 = list(combinations(list(range(len(grupo_tomas_4))),n))
                # ídem para el 4º día
                for sesion_4 in sesiones_4:
                    sesion_indices_4 = get_indices_sesion(sesion_4, grupo_tomas_4)
                    coste_sesion_4 = sesion_cost(sesion_indices_4, tomas)  
                    grupo_tomas_5 = np.delete(grupo_tomas_4, sesion_4, axis=0)
                    # en realidad el 5º dia nos quedan 6 tomas, por lo que la combinación será única
                    sesiones_5 = list(combinations(list(range(len(grupo_tomas_5))),n))
                    sesion_indices_5 = get_indices_sesion(sesiones_5, grupo_tomas_5)
                    # por consistencia con el resto de sesiones
                    sesion_indices_5 = sesion_indices_5[0]
                    coste_sesion_5 = sesion_cost(sesion_indices_5, tomas)  
                    # coste completo de un plan de rodaje de 5 sesiones 
                    coste_plan = coste_sesion + coste_sesion_2 + coste_sesion_3 + coste_sesion_4 + coste_sesion_5
                    # combino las sesiones en un array 2D
                    plan_rodaje = np.row_stack((sesion_indices,sesion_indices_2,sesion_indices_3,sesion_indices_4,sesion_indices_5))
                    i+=1
                    if coste_plan < mejor_coste_plan:
                        # actualizo el mejor coste
                        mejor_coste_plan = coste_plan
                        # actualizo el mejor plan
                        mejor_plan = plan_rodaje.copy()
                        current_time = time.time()
                        elapsed_time = np.round((current_time - start_time),1)
                        print("Combinación (",i,":",elapsed_time,"seg.)",
                        "\nmejor plan:\n",mejor_plan,
                        "\nmejor coste = ", mejor_coste_plan, " salarios")   
    return mejor_plan

Teniendo ya creadas las diversas funciones auxiliares y basándome en lo descrito anteriormente, desarrollo bajo python un posible algoritmo de fuerza bruta que iría creando cada uno de los posibles planes de rodaje (planes de 5 sesiones de 6 tomas distintas) y evaluando el coste total de cada uno de ellos:

Finalmente, el algoritmo se ejecutaría con la siguiente llamada:

In [ ]:
Mejor_Plan_rodaje(tomas)

Obviamente, el tiempo necesario para alcanzar la solución más óptima tras evaluar cada uno de los planes sería prohibitivo. Aún así y a modo ilustrativo, incluyo una muestra de los planes de trabajo que el algoritmo va encontrando en su búsqueda, cada uno algo más óptimo que el anterior en cuanto a coste. Puede verse que el tiempo necesario para ir alcanzando estas planes se va incrementando en buena manera:

EJEMPLO DE SALIDA:
```
Combinación ( 772634 : 51.5 seg.) 
mejor plan:
 [[ 1  2  3  4  5  6]
 [ 7  8  9 10 11 12]
 [13 14 17 18 20 27]
 [15 16 22 25 28 30]
 [19 21 23 24 26 29]] 
mejor coste =  32  salarios
Combinación ( 1479529 : 97.8 seg.) 
mejor plan:
 [[ 1  2  3  4  5  6]
 [ 7  8  9 10 11 12]
 [13 14 20 26 27 29]
 [15 16 22 25 28 30]
 [17 18 19 21 23 24]] 
mejor coste =  31  salarios
```

En este ejemplo, tras unos minutos se ha llegado a un *plan de trabajo* de coste 31, que si bien no sería una solución, al menos mejora el coste máximo de 50 (todos los actores presentes cada día), por lo que cierta optimización a alcanzado.

## Calcula la complejidad del algoritmo por fuerza bruta

Respuesta

Tal como se ha descrito al principio, el algoritmo de fuerza bruta ha de componer todos los planes posibles de rodaje y evaluar su coste. Para ello ha de comenzar iterando sobre todas las posibles combinaciones de sesiones de 6 tomas dentro de las 30 disponibles el primer día, e ir componiendo de manera recursiva (o anidada) el plan de rodaje para los sucesivos días. Cada día el número de tomas disponibles se irá reduciendo al no ser necesario incluir las tomas ya elegidas los días previos. Aún así el número total de posibles planes de rodajes es altísimo y viene dado inicialmente por el siguiente producto de suscesivas combinaciones:

$$ {30\choose 6}{24\choose 6}{18\choose 6}{12\choose 6}{6\choose 6} = \frac{30!}{(6!)^{5}} $$

Que tras simplicar numeradores y denominadores, e introducir un factor que corrige el hecho de que algunos *planes de rodaje* salgan repetidos (mismas sesiones aunque ordenadas en diferentes días), nos queda como:

$$ N_{planes} = \frac{30!}{(6!)^{5}{5!}} = 1.142395e+16 $$

Si bien tanto el numerador como denominador incluyen cantidades factoriales, está claro que el numerador será preponderante y podemos concluir a priori que que el algoritmo tendrá un **orden de complejidad factorial**:

$$ O(n!) $$

El algoritmo de fuerza bruta no solo parte inicialmente de un número muy elevado de sets de tomas posibles, sino que ha de ser implementado de manera recursiva, y aunque cada recursión se realiza sobre un número n decreciente a medida que los sets de tomas usados se van retirando del conjunto de sets original, los bucles siguen teniendo una dependencia factorial, lo cual lo convierta posiblemente en un problema de los llamando **NP-hard**

Como curiosidad he calculado a *grosso modo* el tiempo que mi computador tarda en alcanzar la última solución del ejemplo ilustrativo (aldededor de *100 segundos* para la combinación número *1.5e+6*). Asumiendo esa pauta, para recorrer y evaluar mediante fuerza bruta los 1.14e+16 planes de rodaje posibles, necesitaríamos un tiempo total de aproximadamente *24000* años:

$$ t_{aprox} = ((((1.14e+16*100/1.5e+6)/3600)/24)/365) \approx 24099\ años $$

## (*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

Debido al casi inabarcable espacio de planes, al menos usando fuerza bruta, haré uso de alguno de los algoritmos metaheúristicos que me permita obtener alguna solución razonablemente buena. Un aspecto que me interesa de dichos algoritmos es que se haga un buen balance entre la *intensificación* y la *diversificación*, por lo que podríamos considerar algoritmos de tipo multi-arranque como el **GRASP**, o algoritmos como el de búsqueda **Tabú** que permiten escapar de mínimos locales y buscar a partir de zonas a priori peores (aquí podría incluir también el de **Colonia de hormigas** o **Recocido simulado**, donde en algún momento dado las planes peores se pueden tener en cuenta como punto de partida para nuevos mínimos). 

Por simplicidad, no consideraré en este caso un algoritmo de tipo **Genético**, ya que si bien tienen un amplio ámbito de aplicabilidad, en este caso el modelado resulta algo complejo ya que dicho tipo de algoritmo me generaría con mucha frecuencia resultados que no serán factibles, es decir, planes de rodaje con sesiones en las que habría tomas repetidas, con lo cual sería necesario introducir algún tipo de función o mecanismo que evalura la viabilidad de un resultado y lo descartara (o modificara) a fin de que se ajustara a las condiciones de una solución (es decir, que cubriera el total de 30 tomas sin repetir ninguna). Esto añadiría más complejidad en la implementación del algoritmo.

He decidido probar un algoritmo de **RECOCIDO SIMULADO (Simulated Annealing)**, ya que me permitirá tantear el espacio de planes en diferentes regiones con cierto nivel de aleatoriedad (diversificación) aunque algo más controlado que si probara únicamente con un algoritmo de *Búsqueda aleatoria*. En cualquier caso, para generar una solución de partida, si que haré uso de un simple algortimo de **búsqueda aleatoria** sobre un número no muy grande de iteraciones.

Defino a continuación una serie de funciones auxiliares que usaré dentro del algoritmo principal:


In [ ]:
# Función que genera una solución viable: un plan de rodaje de 5 sesiones de 6 tomas diferente diarias
def genera_plan(tomas):
    grupo_tomas = np.arange(30)
    plan = []
    sesion = []
    coste_plan = 0
    for i in range(5):
        # elige 6 tomas diferentes
        sesion = np.random.choice(grupo_tomas, 6, replace=False)
        # obten los ID's de las tomas que conforman la sesión
        sesion_id = get_indices_sesion(sesion,tomas)
        # calcular el coste salaria de la sesión
        coste_sesion = sesion_cost(sesion_id, tomas)
        # calcular el coste acumulado del pla de rodaje
        coste_plan = coste_plan + coste_sesion
        # ir creando el plan de rodaje
        plan.append(sesion_id)
        # retirar las tomas ya ultilizadas
        grupo_tomas = np.setdiff1d(grupo_tomas, sesion)
    # concateno las sesiones en un array 1D
    plan = np.array(plan).flatten()
    return plan, coste_plan

In [ ]:
# Función para generar una solución inicial de partida usando Búsqueda Aleatoria
def busqueda_aleatoria_plan(tomas, n):
    # por defecto, costé máximo asumiendo necesitara los 10 actores cada día
    mejor_coste = 50
    mejor_plan = []
    for i in range(n):
        plan, coste = genera_plan(tomas)
        if coste < mejor_coste:
            mejor_coste = coste
            mejor_plan = plan
    return mejor_plan, mejor_coste

In [ ]:
# Función para generar una solución vecina intercambiando 2 tomas aleatoriamente
def genera_plan_vecino(plan):
  plan = plan.tolist()
  # Se eligen dos nodos aleatoriamente
  i,j = sorted(random.sample( range(1,len(plan)) , 2))
  # Devuelve una nueva solución pero intercambiando los dos nodos elegidos al azar
  plan_vecino = plan[:i] + [plan[j]] + plan[i+1:j] + [plan[i]] + plan[j+1:]
  # la devuelvo como array en vez de lista, para usarla en mis otras funciones
  return np.array(plan_vecino)

In [ ]:
# Funcion de probabilidad para aceptar peores soluciones
def probabilidad(T,d):
  if random.random() <  math.exp( -1*d / T)  :
    return True
  else:
    return False

# Funcion de descenso de temperatura
def bajar_temperatura(T):
  return T*0.99

Implemento a continuación el algoritmo principal usando la técnica del **RECOCIDO SIMULADO**:

In [ ]:
### ALGORITMO PRINCIPAL DE RECOCIDO SIMULADO ###
def recocido_simulado(plan_referencia, coste_plan_referencia, tomas, temperatura):
  # plan_referencia = plan inicial
  # coste_plan_referencia = coste del plan inicial
  # tomas = datos sobre cada toma
  # temperatura = Temperatura
  
  mejor_plan = []
  mejor_coste = 10e100
  
  n=0
  while temperatura > .0001:
    n+=1
    #Genera una solución vecina
    #vecina =genera_vecina_aleatorio(solucion_referencia)
    plan_vecino = genera_plan_vecino(plan_referencia)
    
    #Calcula su valor(distancia)
    coste_plan_vecino = plan_cost(plan_vecino.reshape(5,6), tomas)
      
    #Si es la mejor solución de todas se guarda(siempre!!!)
    if coste_plan_vecino < mejor_coste:
        mejor_plan = plan_vecino.copy()
        mejor_coste = coste_plan_vecino
    
    #Si la nueva vecina es mejor se cambia  
    #Si es peor se cambia según una probabilidad que depende de T y delta(distancia_referencia - distancia_vecina)
    if coste_plan_vecino < coste_plan_referencia or probabilidad(temperatura, abs(coste_plan_referencia - coste_plan_vecino)) :
      #solucion_referencia = copy.deepcopy(vecina)
      plan_referencia = plan_vecino.copy()
      coste_plan_referencia = coste_plan_vecino

    #Bajamos la temperatura
    temperatura = bajar_temperatura(temperatura)

  return mejor_plan, mejor_coste

## (*)Calcula la complejidad del algoritmo 

Respuesta

Sin considerar la complejidad del cálculo inicial de la solución de entrada (en el que uso un algoritmo de *búsqueda aleatoria*, en el caso del algoritmo de *recocido* tengo por un lado que el bucle *while* principal es manejado por el resultado dado por la función *bajar_temperatura*, que en este caso particular es del tipo *descenso exponencial*, por lo que intuye en esta componente un coste de $O(log(n))$. Por otro parte, dentro del bucle realizo repetidamente una serie de operaciones, entre ellas la más importante quizás sea la generación de la solución vecina, que tiene un coste de $ O(n^2) $ y el cálculo del *plan cost* con un coste $ O(n) $. Creo que entonces el coste aproximado del algoritmo podría ser muy similar a:
$$ O((n^2 + n)*log(n)) $$

## Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta

Haré uso del algoritmo de búsqueda aleatoria a fin de generar rápidamente un *plan de rodaje* aleatorio, válido y relativamente *bueno*, que me servirá de entrada para el algoritmo de *Recocido*. Dicho plan de rodaje consiste en 5 sesiones/día de 6 tomas cada una, en la que ninguna toma se repite y por otro lado el total de tomas cubre las 30 requeridas para completar el rodaje:

In [ ]:
 # creo una solución inicial relativamente buena usando la búsqueda aleatoria con pocas iteraciones
 plan_referencia, coste_plan_referencia = busqueda_aleatoria_plan(tomas, 1000)
 print("Plan inicial:\n\n", plan_referencia.reshape(5,6))
 print("\nCoste plan inicial:\n\n", coste_plan_referencia, " salarios")

Plan inicial:

 [[19 27  6 15  1 29]
 [10  4 21  3  8 11]
 [13 16 30  7 25 20]
 [24 12  9  5 22 28]
 [17  2 26 18 14 23]]

Coste plan inicial:

 33  salarios


## Aplica el algoritmo al juego de datos generado

Respuesta

Aplicando el algoritmo de *recocido* consigo mejorar el plan de rodaje inicial y conseguir uno con un coste salarial algo mejor. En este caso he ejecutado el algoritmo unas pocas veces y he conseguido un plan con un coste total de **28 salarios**, lo cual supondría **un ahorro de hasta el 44%** con respecto a convocar a los 10 actores cada día de doblaje. Dicho plan muestra las tomas que se han de rodar en la sesión de cada día (las tomas por sesión son independientes de su orden):

In [ ]:
mejor_plan, coste_mejor_plan  = recocido_simulado(plan_referencia, coste_plan_referencia, tomas, 10000000)
print("Plan de rodaje por días:\n\n", mejor_plan.reshape(5,6))
print("\nCoste:\n\n", coste_mejor_plan, " salarios")

Plan de rodaje por días:

 [[19  3  9  6  1  7]
 [ 2 20 27 13 17 23]
 [14 12 22 28 18  8]
 [ 5 15 16 30  4 25]
 [24 21 11 26 29 10]]

Coste:

 28  salarios


## Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

1. Long, Xiaoqing & Jinxing, Zhao. (2020). Scheduling Problem of Movie Scenes Based on Three Meta-Heuristic Algorithms. IEEE Access. PP. 1-1. 10.1109/ACCESS.2020.2982664. 
2. Yulian Liu, Qiuji Sun, Xiaotian Zhang, Yiwei Wu, "Research on the Scheduling Problem of Movie Scenes", Discrete Dynamics in Nature and Society, vol. 2019, Article ID 3737105, 8 pages, 2019. https://doi.org/10.1155/2019/3737105
3. T. Cheng, J. Diamond, and B. Lin. Optimal scheduling in film production to minimize talent hold cost. Journal of Optimization Theory and Applications, 79(3):479–492, 1993.
4. Sean Luke, 2013, Essentials of Metaheuristics, Lulu, second edition, available for free at http://cs.gmu.edu/~sean/book/metaheuristics/
5. "Combination of splitting elements into pairs". Entrada en Stackexchage: https://math.stackexchange.com/questions/35684/combination-of-splitting-elements-into-pairs

## Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta

Ciertamente el problema podría ampliarse introduciendo nuevas condiciones, por ejemplo diferentes salarios para según que actor. En tal caso la función objetivo o de coste habría de ser evaluada teniendo en cuenta que actores específicos estarían presentes en cada toma y por ende, en cada sesión diaria.

Otro factor que se podría considerar a fin de extender el problema, sería el introducir una programación temporal de las tomas, o una disponibilidad horaria de cada actor. En ambos casos esto conduciría a que las tomas en cada sesión diaria tendrían que ser conducidas en un orden estricto lo cual aumentaría el espacio de soluciones (en el caso actual, al no importarnos el orden hemos tomado combinaciones de tomas diarias en lugar de permutaciones donde importe el orden).